<a href="https://colab.research.google.com/github/osipov/edu/blob/master/tf0/Demo_Tensorflow_kerasSequential_Classifier.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# building blocks of our network
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

tf.random.set_seed(42);

In [ ]:
NUM = 100

#X spam data points 

#lower left
Xs1 = tf.random.normal((NUM // 2, 2), mean=0, stddev=3) - 5
#upper right
Xs2 = tf.random.normal((NUM // 2, 2), mean=0, stddev=3) + 5

#100 spam data points
Xs = tf.concat([Xs1, Xs2], axis=0)


#X not spam data points
#upper left
Xns1 = tf.Variable(tf.random.normal((NUM // 2, 2), mean=0, stddev=3) + 5)
Xns1[:, 1].assign(Xns1[:, 1] - 10)

#lower right
Xns2 = tf.Variable(tf.random.normal((NUM // 2, 2), mean=0, stddev=3) - 5)
Xns2[:, 1].assign(Xns2[:, 1] + 10)

#100 not spam data points
Xns = tf.concat([Xns1, Xns2], axis=0)

#200 data points, 100 spam, 100 not spam
X = tf.concat([Xs, Xns], axis=0)
plt.figure(figsize = (16, 9))

plt.scatter(X[:NUM, 0], X[:NUM, 1], color = 'orange');
plt.scatter(X[NUM:, 0], X[NUM:, 1], color = 'blue');

plt.xlim([-10, 10])
plt.ylim([-10, 10]);

In [ ]:
X.shape

In [ ]:
# #spam = 1, not spam = 0
y = tf.concat( [tf.ones(NUM, dtype=tf.int64), tf.zeros(NUM, dtype=tf.int64)], axis=0 )
y

In [ ]:
model = Sequential()
model.add(Dense(units = 2, input_shape=(2,), activation='relu'))
model.add(Dense(units = 3, activation='relu'))
model.add(Dense(units = 2, activation='softmax'))

def forward(X, training=True):
    # effectively log softmax
    return tf.math.log(model(X, training))

def loss(y_pred, y):
    '''
    Negative log likelihood (unweighted)
    '''
    return -tf.math.reduce_mean(tf.one_hot(y, depth=2) * y_pred, axis = None)

def metric(y_pred, y):
    return (1. / len(y)) * tf.math.count_nonzero(tf.equal(tf.argmax(y_pred, axis=1),y), axis=0, dtype=tf.float32)

In [ ]:
LEARNING_RATE = 0.002
optimizer = Adam(learning_rate = LEARNING_RATE)

y_pred = forward(X)
loss(y_pred, y), metric(y_pred, y)

In [ ]:
model.summary()

In [ ]:
EPOCHS=1_000
for epoch in range(EPOCHS):
    with tf.GradientTape() as tape:
        # forward pass
        y_pred = forward(X, training=True)

        # compute loss
        last_loss = loss(y_pred, y)

    # back propagation
    grads = tape.gradient(last_loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    # monitor performance
    loss_val, accuracy = float(last_loss), metric(y_pred,y).numpy()
    if (epoch % 100 == 0):
        print(f'epoch {epoch+1:2d}:  loss={loss_val:7.4f}, Accuracy={accuracy:7.4f}')

In [ ]:
x_min, x_max, y_min, y_max, step = -15., 15, -15., 15., 0.5

xx, yy = tf.meshgrid(tf.range(x_min, x_max, step),
                    tf.range(y_min, y_max, step))

X_test = tf.concat([tf.reshape(xx, [tf.size(xx), 1]), 
                tf.reshape(yy, [tf.size(yy), 1])], 
               axis = 1)
y_test = tf.argmax(forward(X_test), axis=1).numpy()

Z = -1 * tf.reshape(y_test, xx.shape)

fig = plt.figure()
plt.axis("equal")
plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.4);
plt.scatter(X[:NUM, 0], X[:NUM, 1], color = 'orange');
plt.scatter(X[NUM:, 0], X[NUM:, 1]);

Copyright 2021 CounterFactual.AI LLC. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.